# Data Preprocessing_ver2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import os, glob, skimage, dataset, cv2
from skimage import io, transform
from tqdm import tqdm
from copy import copy
from tools import *

%matplotlib inline

In [2]:
img_path = explore_dir('./dataset',"nii.gz")

train_indices = list()
test_indices = list()

csv_file = pd.read_csv("./S1_Data.csv")

for i in range(len(list(csv_file.Holdout))):
    if csv_file.Holdout[i]=='Train':
        train_indices.append(i)
    else:
        test_indices.append(i)

In [3]:
csv_file = csv_file.as_matrix()

train_csv = csv_file[train_indices[0]][np.newaxis,:]
test_csv = csv_file[test_indices[0]][np.newaxis,:]

for i, entity in enumerate(csv_file):
    if i in train_indices[1:] :
        train_csv = np.concatenate((train_csv,[entity]),axis=0)
    elif i in test_indices[1:] :
        test_csv = np.concatenate((test_csv,[entity]),axis=0)

In [4]:
train_dataset = list()
train_labels = list()
test_dataset = list()
test_labels = list()

with tqdm(total=len(img_path[0])) as pbar:
    pbar.set_description('image_load')
    for i, nibFilename in enumerate(img_path[0]):
        brain = apply_3D_bbox(np.array(nib.load(img_path[1][i]).get_data(), dtype=np.float64))
        if nibFilename in train_csv[:,7]:
            train_dataset.append(transform.resize(brain,(64,64,64)))
            train_labels.append(train_csv[list(train_csv[:,7]).index(nibFilename),10])
        elif nibFilename in test_csv[:,7]:
            test_dataset.append(transform.resize(brain,(64,64,64)))
            test_labels.append(test_csv[list(test_csv[:,7]).index(nibFilename),10])
        pbar.update(1)

image_load:   0%|          | 0/5067 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
image_load: 100%|██████████| 5067/5067 [27:55<00:00,  3.30it/s]
